In [1]:
import subprocess
import os
import sys
import pandas as pd
import wikiapi
import re
from collections import defaultdict
import gensim
import requests
from string import punctuation
from lxml import html
import random
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import svm
import numpy as np
from googletrans import Translator
from gensim.models.keyedvectors import KeyedVectors
from nltk.corpus import wordnet as wn

In [5]:
class Coreftrain:
    
    def __init__(self):
        self.docs = pd.read_csv('coref/Documents.txt', delimiter='\t')
        self.train_noun = pd.DataFrame(columns=['docid', 'start_ref', 'cor', 'incor'])
        self.gold_noun =  pd.DataFrame(columns=['docid', 'start_ref', 'refs'])
        self.no_gold_noun =  pd.DataFrame(columns=['docid', 'start_ref', 'refs'])
        self.train_ne = pd.DataFrame(columns=['docid', 'start_ref', 'cor', 'incor'])
        self.gold_ne =  pd.DataFrame(columns=['docid', 'start_ref', 'refs'])
        self.no_gold_ne =  pd.DataFrame(columns=['docid', 'start_ref', 'refs'])
        self.train_pron = pd.DataFrame(columns=['docid', 'start_ref', 'cor', 'incor'])
        self.gold_pron =  pd.DataFrame(columns=['docid', 'start_ref', 'refs'])
        self.no_gold_pron =  pd.DataFrame(columns=['docid', 'start_ref', 'refs'])
        self.answers_noun = pd.DataFrame() # hold answers for training sets. separtatedby type. 9 total.
        self.answers_ne = pd.DataFrame()
        self.answers_pron = pd.DataFrame()
        self.train_var_noun = pd.DataFrame()#training data
        self.train_var_ne = pd.DataFrame()
        self.train_var_pron = pd.DataFrame()
        self.gold_var_noun = pd.DataFrame()#start vars for nouns
        self.no_gold_var_noun = pd.DataFrame()
        self.gold_var_ne = pd.DataFrame() #start vars for NEs
        self.no_gold_var_ne = pd.DataFrame()
        self.gold_var_pron = pd.DataFrame()#start vars for pronouns
        self.no_gold_var_pron = pd.DataFrame()
        self.idstr_noun = pd.DataFrame() # holds ids for merge reference.
        self.idstr_ne = pd.DataFrame()
        self.idstr_pron = pd.DataFrame()
        self.ids_ne_gold = pd.DataFrame()# ne ids
        self.ids_ne_nogold = pd.DataFrame()
        self.ids_noun_gold = pd.DataFrame()# ne noun
        self.ids_noun_nogold = pd.DataFrame()
        self.ids_pron_gold = pd.DataFrame()# ne pron
        self.ids_pron_nogold = pd.DataFrame()
        
    
        #merge functions from previous doc. Was decided that corefsetup already to large, hence the repetition.
    
    def prepdf(self):
        '''
        Specifc to the structure of rucoref data. Creates table with doc ids and names.
        '''          
        self.newdf = pd.DataFrame(columns=['docid', 'docname'])
        for path in self.docs['path']:
            name = path.split('/')[1]
            did = self.docs.loc[self.docs.path.values == path, 'doc_id'].values[0]
            self.newdf = self.newdf.append(pd.Series([did, name], index=['docid', 'docname']), ignore_index=True)
        return self.newdf





    def old_morph(self, headmen, head_pos):
        '''
        gathers morphological info from head mention.
        '''
        leng = len(headmen['morph'].values[0].split('|'))
        if leng == 4:
            ment_anim = headmen['morph'].values[0].split('|')[0].replace('Animacy=', '').split(',')
            ment_case = headmen['morph'].values[0].split('|')[1].replace('Case=', '') #only one possible value.
            ment_gen = headmen['morph'].values[0].split('|')[2].replace('Gender=', '').split(',')
            ment_num = headmen['morph'].values[0].split('|')[3].replace('Number=', '').split(',')
        elif leng == 3:
            ment_anim = None
            ment_case = headmen['morph'].values[0].split('|')[0].replace('Case=', '') #only one possible value.
            ment_gen = headmen['morph'].values[0].split('|')[1].replace('Gender=', '').split(',')
            ment_num = headmen['morph'].values[0].split('|')[2].replace('Number=', '').split(',')
        else:
            ment_anim = None
            ment_case = None
            ment_gen = None
            ment_num = None
        return ment_anim, ment_case, ment_gen, ment_num
    
    
    def new_morph(self, new_anim, new_gen, new_num, ment_anim, ment_gen, ment_num, mer_men):
        '''
        combines morphological features of mentions with head mention.
        '''
        if not ment_anim == None:
            if len(mer_men['morph'].values[0].split('|')) == 4:
                mer_anim = mer_men['morph'].values[0].split('|')[0].replace('Animacy=', '').split(', ')
                mer_gen = mer_men['morph'].values[0].split('|')[2].replace('Gender=', '').split(', ')
                mer_num = mer_men['morph'].values[0].split('|')[3].replace('Number=', '').split(', ')
                for mer_a in mer_anim:
                    if mer_a not in ment_anim: #merging morphological features, if different.
                        new_anim.append(mer_a)
                for mer_g in mer_gen:
                    if mer_g not in ment_gen:
                        new_gen.append(mer_g)
                for mer_n in mer_num:
                    if mer_n not in ment_num:
                        new_num.append(mer_n)
                no_merge = False
            else:
                no_merge = True
        elif not ment_gen == None:
            mer_gen = mer_men['morph'].values[0].split('|')[1].replace('Gender=', '').split(', ')
            mer_num = mer_men['morph'].values[0].split('|')[2].replace('Number=', '').split(', ')
            new_anim = None
            for mer_g in mer_gen:
                if mer_g not in ment_gen:
                    new_gen.append(mer_g)
            for mer_n in mer_num:
                if mer_n not in ment_num:
                    new_num.append(mer_n)
            no_merge = False
        else:
            new_anim = None
            new_gen = None
            new_num = None
            no_merge = True
        return new_anim, new_gen, new_num, no_merge
          

# merge types
        
   
    def morph_merge(self, headmen, merges, mens):
        '''
        merges morphology of mentions to head mention.
        '''
        head_pos = headmen['udpos']
        ment_anim, ment_case, ment_gen, ment_num = self.old_morph(headmen, head_pos) #creates list of all morph features of head mention.
        new_anim = []
        new_gen = []
        new_num = []
        if not ment_anim == None:
            new_anim.extend(ment_anim)
        elif not ment_case == None:
            new_gen.extend(ment_gen)
            new_num.extend(ment_num)
        for mer in merges:
            mer_men = mens[mens['shift'] == mer]
            new_anim, new_gen, new_num, no_merge = self.new_morph(new_anim, new_gen, new_num, ment_anim, ment_gen, ment_num, mer_men)
        if not new_anim == None and no_merge == False:
            new_anim = ", ".join(list(set(new_anim))) #remove repeating features.
            new_gen = ", ".join(list(set(new_gen)))
            new_num = ", ".join(list(set(new_num)))
            new_mor = 'Animacy={0}|Case={1}|Gender={2}|Number={3}'.format(new_anim, ment_case, new_gen, new_num)
        elif no_merge == False and not new_gen == None:
            new_gen = ", ".join(list(set(new_gen)))
            new_num = ", ".join(list(set(new_num)))
            new_mor = 'Case={0}|Gender={1}|Number={2}'.format(ment_case, new_gen, new_num)
        else:
            new_mor = headmen['morph'].values[0]
        return new_mor   
    
    
    def sid_merge(self, headmen, headmen_id, merges, mens):
        '''
        merges sentence id info, for later calculation of distances, as program chooses closest distance
        when considering sentence range. Creates clust_head category for referencing back to head in order to
        gather cluster level info.
        '''
        head_sid = headmen['sid'].values #extract all values
        new_sids = []
        new_sids.extend(head_sid)
        clust_head_of_head = mens.ix[headmen_id, 'clust_head'] # find head of head mention.
        for mer in merges:
            mer_men = mens[mens['shift'] == mer]
            mer_sid = mer_men['sid'].values
            mer_id = mer_men.index.tolist()[0] #check functioning!
            new_sids.extend(mer_sid)
            if not clust_head_of_head == '_': # adds head_clust of headmen to mention if it exists.
                mens.ix[mer_id, 'clust_head'] = clust_head_of_head
            else:
                mens.ix[mer_id, 'clust_head'] = headmen['shift'].values[0] #shift head
        new_sids = ', '.join(list(set([str(x) for x in new_sids])))
        mens.ix[headmen_id, 'sid_corefs'] = new_sids
        return mens
            
            
            
    def merge(self, mens, men_dict, manual, morph_merge, gold): # main merge function.
        '''
        merges data for corefering mentions(i.e. sid, shift of all), appending necessary 
        info to head mention (often most informative mention.), removes all child 
        mentions from mention list, replaces child mention pairings in candidate list with head word. 
        Because features are aggregated across all mentions in cluster, will not make impact on performance.
        '''      
        if manual == True:
            for ment, merges in men_dict.items():
                skip = False
                headmen = mens[mens['shift'] == ment]
                if gold == True: #to select for merging in gold model, only those that exist.
                    indexes = []
                    if not headmen.empty:
                        indexes.append(ment)
                    for mer in merges:
                        mermen = mens[mens['shift'] == mer]
                        if not mermen.empty:
                            indexes.append(mer)
                    if len(indexes) > 0:
                        indexes.sort() # to get the most early occuring mention first in order.
                        head_sh = indexes[0]
                        headmen = mens[mens['shift'] == head_sh]
                        headmen_id = headmen.index.tolist()[0]
                        del indexes[0]
                        merges = indexes 
                    else:
                        skip = True
                else:
                    try: #because of trying to train mens and deps at same time
                        headmen_id = headmen.index.tolist()[0]
                    except IndexError:
                        print('first ment error!', ment)               
                if skip == False:
                    mens.ix[headmen_id, 'corefs'] = ', '.join([str(x) for x in merges]) # add shifts of all corefering mentions
                    if len(merges) > 0:
                        if morph_merge == True:
                            new_mor = self.morph_merge(headmen, merges, mens)
                            mens.ix[headmen_id, 'morph'] = new_mor
                        mens = self.sid_merge(headmen, headmen_id, merges, mens) 
                    mens = mens.fillna('_')
            return mens  
    
    
    def form_frame(self, deps, mens, type_all, l_sids, r_sids, sid, typ, goldm):
        if type_all == 'pron':
            if typ in ('pron', 'poss'):
                rng = 3
                frame = self.create_frames(l_sids, r_sids, sid, rng, deps, goldm, mens)#sentence frames for candidate searching.
            elif typ in ('refl', 'rel'):
                if goldm == False:
                    frame = deps[deps['sid'] == sid]
                else:
                    frame = mens[mens['sid'] == sid]
            else:
                frame = []
        elif type_all == 'ne-ne':
            if typ == 'NE':
                rng = 4
                frame = self.create_frames(l_sids, r_sids, sid, rng, deps, goldm, mens)
            else:
                frame = []
        elif type_all == 'ne-noun':
            if typ == 'NE':
                rng = 4
                frame = self.create_frames(l_sids, r_sids, sid, rng, deps, goldm, mens)
            else:
                frame = []
        elif type_all == 'noun-noun':
            if typ == 'noun':
                rng = 4
                frame = self.create_frames(l_sids, r_sids, sid, rng, deps, goldm, mens)
            else:
                frame = [] #catch gold mens to deps descrepancies for training set. Just skip.
        return frame
    
            
    def set_up_frames(self, ment, typ, type_all, deps, mens, train, goldm):  
        '''
        sort frame ranges and frames for different mentions.
        '''
        sid = int(ment['sid'])
        sid_max = int(deps['sid'][deps['sid'].idxmax()])
        ment_sh = ment['shift']
        l_sids = 0
        r_sids = 0
        if sid < 4: #finds left and right boundaries if sentence at beginning or end of doc.
            l_sids = [x for x in range(1, sid+1)]
        elif (sid_max - sid) < 4:
            r_sids = [x for x in range(sid, sid_max+1)]
        frames = self.form_frame(deps, mens, type_all, l_sids, r_sids, sid, typ, goldm)
        return frames
    

    def create_frames(self, l_sids, r_sids, sid, rng, deps, goldm, mens):
        '''
        creates frames for mention search.
        '''
        if goldm == False:
            if not l_sids == 0 and not r_sids == 0:
                l_len = len(l_sids)
                r_len = len(r_sids)
                if l_len > 0 and r_len > 0:
                    frame = deps.query('{0} <= sid <= {1}'.format(l_sids, r_sids))
                elif l_len > 0 and not r_len > 0:
                    frame = deps.query('{0} <= sid <= {1}'.format(l_sids, sid+rng))
                elif r_len > 0 and not l_len > 0:
                    frame = deps.query('{0} <= sid <= {1}'.format(sid-rng, r_sids))
            else:
                frame = deps.query('{0} <= sid <= {1}'.format(sid-rng, sid+rng))
        else:
            if not l_sids == 0 and not r_sids == 0:
                l_len = len(l_sids)
                r_len = len(r_sids)
                if l_len > 0 and r_len > 0:
                    frame = mens.query('{0} <= sid <= {1}'.format(l_sids, r_sids))
                elif l_len > 0 and not r_len > 0:
                    frame = mens.query('{0} <= sid <= {1}'.format(l_sids, sid+rng))
                elif r_len > 0 and not l_len > 0:
                    frame = mens.query('{0} <= sid <= {1}'.format(sid-rng, r_sids))
            else:
                frame = mens.query('{0} <= sid <= {1}'.format(sid-rng, sid+rng))
        return frame

    
    def create_test_sets(self, ment, deps, frame, docid, df, ment_ser, goldm):
        '''
        appends rows to test sets for each ment
        '''
        ments = []
        ment_sh = ment['shift']
        ment_shs = ment['tk_shifts'].split(', ')
        if not len(frame) == 0: #aviod skipped mentions.
            for i, m in frame.iterrows():
                m_sh = m['shift']
                if m_sh in ment_shs or m_sh == ment_sh: # so not to capture mentions in mentions as possibly coreferent if they overlap at all.
                    pass
                elif not m['full_men'] == '_':
                    ments.append(m_sh)
            ments = list(set(ments))
            tempval = {'docid':[docid], 'start_ref':[ment_sh], 'refs':[ments], 'series':[ment_ser]}
            tempdf = pd.DataFrame(tempval)
            df = df.append(tempdf)
        return df
    
    
    def create_candidates(self, mens, deps, train, gold, no_gold, docid, \
                          train_ids, test_ids, type_all):
        '''
        creates initial candidate lists for training, gold mentions, and no gold mentions.
        '''
        if docid in test_ids:
            for i, ment in deps.iterrows():
                if not ment['full_men'] == '_' and not ment['udpos'] == 'PUNCT': #avoid non mentions and parsing errors.
                    ment_typ = ment['type']
                    if 'series' not in ment:
                        ment['series'] = '_'
                    ment_ser = ment['series']
                    frame = self.set_up_frames(ment, ment_typ, type_all, deps, mens, train=False, goldm=False)
                    no_gold = self.create_test_sets(ment, deps, frame, docid, no_gold, ment_ser, goldm=False)
        for i, ment in mens.iterrows():
            if not ment['udpos'] == 'PUNCT' and not ment['full_men'] =='_':
                if 'series' not in ment:
                    ment['series'] = '_'
                if docid in test_ids:
                    ment_typ = ment['type']
                    ment_ser = ment['series']
                    frame = self.set_up_frames(ment, ment_typ, type_all, deps, mens, train=False, goldm=True)
                    gold = self.create_test_sets(ment, deps, frame, docid, gold, ment_ser, goldm=True)
                else:
                    sid = int(ment['sid'])
                    ment_sh = ment['shift']
                    ment_typ = ment['type']
                    ment_ser = ment['series']
                    frame = self.set_up_frames(ment, ment_typ, type_all, deps, mens, train=True, goldm=False) # create frame.
                    if len(frame) == 0:
                        pass
                    else:
                        cor_id = ment['chain_id']
                        cor_men = mens['shift'][mens['chain_id'] == cor_id].tolist()
                        cor_ments = []
                        for cor in cor_men:
                            ment_temp = frame[frame['shift'] == cor]
                            cor_ments.append(ment_temp)
                        n_cor_ments = []
                        for df in cor_ments:
                            if df.empty:
                                pass
                            elif df['shift'].values[0] == ment_sh:
                                pass
                            else:
                                n_cor_ments.append(df)
                        if len(n_cor_ments) == 0: #for refs that can't be found within the given frame.
                            pass
                        else:
                            cor = random.choice(n_cor_ments) # in order to randomly choose mentions.
                            cor_sh = cor['shift'].values[0]
                            if cor_sh > ment_sh: #select incorrect only if it's between correct
                                incors = frame[(ment_sh < frame['shift']) & (frame['shift'] < cor_sh)]
                            elif ment_sh > cor_sh:
                                incors = frame[(cor_sh < frame['shift']) & (frame['shift'] < ment_sh)]
                            incor_ments = []
                            for i, incorr in incors.iterrows():
                                if incorr['shift'] in cor_men or incorr['shift'] == ment_sh:
                                    pass
                                elif not incorr['full_men'] == '_': #weed out non mentions
                                    incor_ments.append(incorr)
                            if len(incor_ments) == 0: # if no possible mentions in between.
                                if cor_sh > ment_sh: #select any possible mention in frame.
                                    incors = frame[(cor_sh < frame['shift']) | (frame['shift'] < ment_sh)]
                                elif ment_sh > cor_sh:
                                    incors = frame[(ment_sh < frame['shift']) | (frame['shift'] < cor_sh)]
                                for i, incorr in incors.iterrows():
                                    if incorr['shift'] in cor_men or incorr['shift'] == ment_sh:
                                        pass
                                    elif not incorr['full_men'] == '_': #weed out non mentions
                                        incor_ments.append(incorr)
                            if len(incor_ments) == 0: # If no other posible mentions in frame besides correct men
                                incors = deps[deps['sid'] == sid + 1] # add following sent so there is a value
                                for i, incorr in incors.iterrows():
                                    if incorr['shift'] in cor_men or incorr['shift'] == ment_sh:
                                        pass
                                    elif not incorr['full_men'] == '_': #weed out non mentions
                                        incor_ments.append(incorr)
                                if len(incor_ments) == 0: # If no other posible mentions in frame besides correct men
                                    incors = deps[deps['sid'] == sid - 1] # add following sent so there is a value
                                    for i, incorr in incors.iterrows():
                                        if incorr['shift'] in cor_men or incorr['shift'] == ment_sh:
                                            pass
                                        elif not incorr['full_men'] == '_': #weed out non mentions
                                            incor_ments.append(incorr)
                            incor = random.choice(incor_ments)
                            incor_sh = incor['shift']
                            tempval = {'docid':[docid], 'start_ref':[ment_sh], 'cor':[cor_sh], 'incor':[incor_sh], 'series':[ment_ser]}
                            tempdf = pd.DataFrame(tempval)
                            train = train.append(tempdf)
        return train, gold, no_gold
      
    
    
    def compare(self, title, val, temp_ref, start_ment, ref_ment):
        '''
        module for comparing values of two mentions. Appends 1 for True, 0 for False to given title.
        '''
        skip = False
        stop = False
        if not start_ment.empty or not ref_ment.empty:
            if val == 'morph': #because need to split morph values.
                start_morph = start_ment[val].values[0].split('|')
                ref_morph = ref_ment[val].values[0].split('|')
                if skip == False:
                    stopg = False # set stops to false default
                    stopa = False
                    stopn = False
                    stopp = False
                    stopgr = False
                    stopar = False
                    stopnr = False
                    stoppr = False
                    for morph in start_morph: # determine existance of morph
                        if title == 'genderm':
                            if morph.startswith('Gender=') and stopg == False:
                                stopg = True
                        elif title == 'animacym':
                            if morph.startswith('Animacy=') and stopa == False:
                                stopa = True
                        elif title == 'numberm':
                            if morph.startswith('Number=') and stopn == False:
                                stopn = True
                        elif title == 'person_match':
                            if morph.startswith('Person=') and stopp == False:
                                stopp = True

                    for morph in ref_morph:
                        if title == 'genderm':
                            if morph.startswith('Gender=') and stopgr == False:
                                stopgr = True
                        elif title == 'animacym':
                            if morph.startswith('Animacy=') and stopar == False:
                                stopar = True
                        elif title == 'numberm':
                            if morph.startswith('Number=') and stopnr == False:
                                stopnr = True
                        elif title == 'person_match':
                            if morph.startswith('Person=') and stoppr == False:
                                stoppr= True


                    if val == 'clustagree':   
                        if stopg == False:
                            start_val_gen = None
                        if stopa == False:
                            start_val_ani = None
                        if stopn == False:
                            start_val_num = None
                        if stopgr == False:
                            ref_val_gen = None
                        if stopar == False:
                            ref_val_ani = None
                        if stopnr == False:
                            ref_val_num = None

                        for morph in start_morph:
                            if morph.startswith('Gender='):
                                start_val_gen = morph
                            if morph.startswith('Animacy='):
                                start_val_ani = morph
                            if morph.startswith('Number='):
                                start_val_num = morph


                        for morph in ref_morph:
                            if morph.startswith('Gender='):
                                ref_val_gen = morph
                            if morph.startswith('Animacy='):
                                ref_val_ani = morph
                            if morph.startswith('Number='):
                                ref_val_num = morph
                        if start_val_gen == ref_val_gen and start_val_ani == ref_val_ani and start_val_num == ref_val_num \
                        and not (start_val_gen == None or start_val_ani == None or start_val_num == None or ref_val_gen == None \
                        or ref_val_ani == None or ref_val_num == None):
                            temp_ref[title] = [1]
                        else:
                            temp_ref[title] = [0]
                        return temp_ref


                    else:
                        if (stopg or stopa or stopa or stopp) == False:
                            start_val = None

                        if (stopgr or stopar or stopar or stoppr) == False:
                            ref_val = None


                        for morph in start_morph:
                            if title == 'genderm':
                                if morph.startswith('Gender='):
                                    start_val = morph
                            elif title == 'animacym':
                                if morph.startswith('Animacy='):
                                    start_val = morph
                            elif title == 'numberm':
                                if morph.startswith('Number='):
                                    start_val = morph
                            elif title == 'person_match':
                                if morph.startswith('Person='):
                                    start_val = morph


                        for morph in ref_morph:
                            if title == 'genderm':
                                if morph.startswith('Gender='):
                                    ref_val = morph
                            elif title == 'animacym':
                                if morph.startswith('Animacy='):
                                    ref_val = morph
                            elif title == 'numberm':
                                if morph.startswith('Number='):
                                    ref_val = morph
                            elif title == 'person_match':
                                if morph.startswith('Person='):
                                    ref_val = morph
                                    
            else: # for all other compares, without special format
                start_val = start_ment[val].values[0]
                ref_val = ref_ment[val].values[0]
        else:
            stop = True

#         if ('start_val' in locals() or 'start_val' in globals()) \
#         and ('ref_val' in locals() or 'ref_val' in globals()):
        
        if not title == 'clustagree':
            if stop == False:
                if ref_val == start_val and not (ref_val or start_val) == None:
                    temp_ref[title] = [1]
                else:
                    temp_ref[title] = [0]
            else:
                temp_ref[title] = [0]

#         else:
#             temp_ref[title] = [0]
        
        return temp_ref
            
            
    def partialheadm(self, title, val, temp_ref, start_ment, ref_ment, deps):
        '''
        relaxed head mention match. If any nouns or proper nouns in mention match returns 1 for true.
        '''
        if not start_ment.empty or ref_ment.empty:
            skip = False
            start_shs = start_ment[val].values[0].split(',')
            ref_shs = ref_ment[val].values[0].split(',')
            if len(ref_shs) > 0:
                for stsh in start_shs:
                    if not stsh == '_':
                        st = deps[deps['shift'] == float(stsh)]
                        st_pos = st['udpos'].values[0]
                        if st_pos in ('NOUN', 'PROPN'): #so that not just pairing anything
                            st_lem = st['lemma'].values[0]
                            for refsh in ref_shs:
                                if not refsh == '_':
                                    rf = deps[deps['shift'] == float(refsh)]
                                else:
                                    skip = True
                            if skip == False:
                                rf_lem = rf['lemma'].values[0]
                                if st_lem == rf_lem:  # compare the two
                                    temp_ref[title] = [1]
                    else:
                        temp_ref[title] = [0]
                if temp_ref[title].empty:
                    temp_ref[title] = [0]
            else:
                temp_ref[title] = [0]
        else:
            temp_ref[title] = [0]
        return temp_ref
                    
        
    def distance(self, title, val, temp_ref, start_ment, ref_ment):
        '''
        finds distance bewteen given start and finish, depending on variable.
        '''
        start_sid = start_ment[val].values[0]
        ref_sid = ref_ment[val].values[0]
        dif = int(start_sid) - int(ref_sid) #can be negative to account for direction.
        temp_ref[title] = [dif]
        return temp_ref
           
        
    def sing_to_plur(self, morph):
        '''
        Change number of series to plural
        '''
        morph = ['Number=Plur' if x=='Number=' else x for x in morph]
        morph = '|'.join(morph)
        return morph
      
        
    def find_val(self, title, val, temp_ref, start_ment, deps, find):
        '''
        returns 1 for true if head begins with a determiner.
        '''
        if val == 'tk_shifts':
            st_first = start_ment[val].values[0]
            if not type(st_first) == float:
                st_first = st_first.split(',')[0]
            if not st_first == '_':
                st_f_pos = deps[deps['shift'] == float(st_first)]['udpos'].values[0]
                if st_f_pos == find:
                    temp_ref[title] = [1]
                else:
                    temp_ref[title] = [0]
            else:
                temp_ref[title] = [0]
        elif val in ('deprel', 'type', 'shift', 'containnum'):
            st_first = start_ment[val].values[0]
            if st_first == find:
                temp_ref[title] = [1]
            else:
                temp_ref[title] = [0]
        return temp_ref
        
        
    def get_val(self, title, val, temp_ref, start_ment):
        temp_ref[title] = [start_ment[val].values[0]]
        return temp_ref
    
    
    def comb_attr(self, title, val, temp_ref, start_ment, ref_ment):
        if not start_ment.empty:
            start_val = start_ment[val].values[0]
            ref_val = ref_ment[val].values[0]
            new_val = '-'.join((start_val, ref_val))
            temp_ref[title] = [new_val]
        else:
            temp_ref[title] = '_'
        return temp_ref
        
    
    def clust_info(self, title, val, temp_ref, start_ment, ref_ment, deps, act):
        '''
        for gathering and comparing cluster level info.
        '''
        ref_head = deps[deps['shift'] == float(ref_ment['head'].values[0])]
        start_head = deps[deps['shift'] == float(start_ment['head'].values[0])]
        if act == 'compare':
            title = title
            val = 'morph'
            temp_ref.merge(self.compare(title, val, temp_ref, start_ment, ref_ment))
        return temp_ref
     
        
    def w2v(self, title, temp_ref, w2vmod, start_ment, ref_ment):
        '''
        retrives word2vec similarity scores for words
        '''
        start_lem = start_ment['lemma'].values[0]
        ref_lem = ref_ment['lemma'].values[0]
        start_pos = start_ment['udpos'].values[0]
        ref_pos = ref_ment['udpos'].values[0]
        start_fin = '_'.join((start_lem, start_pos))
        ref_fin = '_'.join((ref_lem, ref_pos))
        try:
            score = w2vmod.similarity(start_fin, ref_fin)
            temp_ref[title] = score
        except KeyError:
            temp_ref[title] = 0
        return temp_ref
    
    
    def wordnet(self, start_ment):
        '''
        set up word list for checking.
        '''
        translator = Translator()
        trans = translator.translate(start_ment['lemma'].values[0], dest = 'en')
        word = trans.text
        syns = wn.synsets(word)
        new_list = []
        for syn in syns:
            for l in syn.lemmas():
                if l.name() not in new_list:
                    new_list.append(l.name().replace('_', ' '))
        trans = translator.translate(new_list, dest = 'ru')
        ru_list = []
        for tran in trans:
            ru_list.append(tran.text)
        return ru_list
    
    
    def in_list(self, title, val, temp_ref, start_list, ref_ment):
        '''
        check if something is in list. 1 for true.
        '''
        ref_val = ref_ment[val].values[0]
        if ref_val in start_list:
            temp_ref[title] = 1
        else:
            temp_ref[title] = 0
        return temp_ref
    
        
    def make_vars(self, deps, starts, refs, var_list, typ, w2vmod, train):
        '''
        makes vars sets for each mention.
        '''
        ret = False
        all_refs = []#list for temp storing of mention comparisons.
        for start in starts:
            try:
                if start['udpos'].values[0]:
                    wnet_list = self.wordnet(start) #set up wnet list once.
                    for ref in refs:
                        ref_ment = deps[deps['shift'] == ref]
                        temp_ref = pd.DataFrame(columns=var_list)
                        if 'genderm' in var_list:
                            title = 'genderm'
                            val = 'morph'
                            temp_ref.merge(self.compare(title, val, temp_ref, start, ref_ment))
                        if 'numberm' in var_list:
                            title = 'numberm'
                            val = 'morph'
                            temp_ref.merge(self.compare(title, val, temp_ref, start, ref_ment))
                        if 'headm' in var_list:
                            title = 'headm'
                            val = 'lemma'
                            temp_ref.merge(self.compare(title, val, temp_ref, start, ref_ment))
                        if 'partialheadm' in var_list:
                            title = 'partialheadm'
                            val = 'tk_shifts'
                            temp_ref.merge(self.partialheadm(title, val, temp_ref, start, ref_ment, deps))
                        if 'sentdist' in var_list:
                            title = 'sentdist'
                            val = 'sid'
                            temp_ref.merge(self.distance(title, val, temp_ref, start, ref_ment))
                        if 'shiftdist' in var_list:
                            title = 'shiftdist'
                            val = 'shift'
                            temp_ref.merge(self.distance(title, val, temp_ref, start, ref_ment))
                        if 'detmen' in var_list:
                            title = 'detmen'
                            val = 'tk_shifts'
                            find = 'DET'
                            temp_ref.merge(self.find_val(title, val, temp_ref, start, deps, find))
                        if 'detref' in var_list:
                            title = 'detref'
                            val = 'tk_shifts'
                            find = 'DET'
                            temp_ref.merge(self.find_val(title, val, temp_ref, ref_ment, deps, find))
                        if 'nsubjant' in var_list:
                            title = 'nsubjant'
                            val = 'deprel'
                            find = 'nsubj'
                            temp_ref.merge(self.find_val(title, val, temp_ref, ref_ment, deps, find))
                        if 'nsubjmen' in var_list:
                            title = 'nsubjmen'
                            val = 'deprel'
                            find = 'nsubj'
                            temp_ref.merge(self.find_val(title, val, temp_ref, start, deps, find))
                        if 'nar_match' in var_list:
                            if not start['nar'].values[0] == '_':
                                title = 'nar_match'
                                val = 'nar'
                                temp_ref.merge(self.compare(title, val, temp_ref, start, ref_ment))
                        if 'antecedenttype' in var_list:
                            title = 'antecedenttype'
                            val = 'type'
                            temp_ref.merge(self.get_val(title, val, temp_ref, ref_ment))
                        if 'namepartref' in var_list:
                            title = 'namepartref'
                            val = 'deprel'
                            find = 'flat:name'
                            temp_ref.merge(self.find_val(title, val, temp_ref, ref_ment, deps, find))
                        if 'isrefspeaker' in var_list:
                            title = 'isrefspeaker'
                            val = 'shift'
                            find = start['nar'].values[0]
                            temp_ref.merge(self.find_val(title, val, temp_ref, ref_ment, deps, find))
                        if 'deproleant' in var_list:
                            title = 'deproleant'
                            val = 'deprel'
                            temp_ref.merge(self.get_val(title, val, temp_ref, ref_ment))
                        if 'deprolemen' in var_list:
                            title = 'deprolemen'
                            val = 'deprel'
                            temp_ref.merge(self.get_val(title, val, temp_ref, start))
                        if 'deprolecomb' in var_list:
                            title = 'deprolecomb'
                            val = 'deprel'
                            temp_ref.merge(self.comb_attr(title, val, temp_ref, start, ref_ment))
                        if 'samesent' in var_list:
                            title = 'samesent'
                            val = 'sid'
                            temp_ref.merge(self.compare(title, val, temp_ref, start, ref_ment))

                            
                        ref_pos = ref_ment['udpos'].values[0]

                        if typ == 'noun': #noun specific vars
                            if ref_pos in ('NOUN', 'PROPN'):
                                if 'animacym' in var_list and ref_pos in ('NOUN', 'PROPN'):
                                    title = 'animacym'
                                    val = 'morph'
                                    temp_ref.merge(self.compare(title, val, temp_ref, start, ref_ment))
                                if 'clustagree' in var_list: #cluster level info
                                    title = 'clustagree'
                                    val = 'morph'
                                    act = 'compare'
                                    temp_ref.merge(self.clust_info(title, val, temp_ref, start, ref_ment, deps, act))
                                if 'containnum' in var_list: #cluster level info
                                    title = 'containnum'
                                    val = 'udpos'
                                    find = 'NUM'
                                    temp_ref.merge(self.find_val(title, val, temp_ref, ref_ment, deps, find))
                                if 'w2v_sim' in var_list:
                                    title = 'w2v_sim'
                                    temp_ref.merge(self.w2v(title, temp_ref, w2vmod, start, ref_ment))
                                if 'wnet_syn' in var_list:
                                    title = 'wnet_syn'
                                    var = 'lemma'
                                    start_list = wnet_list
                                    temp_ref.merge(self.in_list(title, var, temp_ref, start_list, ref_ment))

                        elif typ == 'ne': #Ne specific vars
                            if ref_pos in ('NOUN', 'PROPN'):
                                if 'animacym' in var_list:
                                    title = 'animacym'
                                    val = 'morph'
                                    temp_ref.merge(self.compare(title, val, temp_ref, start, ref_ment))
                                if 'clustagree' in var_list: #cluster level info
                                    title = 'clustagree'
                                    val = 'morph'
                                    act = 'compare'
                                    temp_ref.merge(self.clust_info(title, val, temp_ref, start, ref_ment, deps, act))
                            if 'NEtype_match' in var_list:
                                title = 'NEtype_match'
                                val = 'ne_type'
                                temp_ref.merge(self.compare(title, val, temp_ref, start, ref_ment))
                            if 'namepart' in var_list: #part of a name, not head. Ideally learns to not pair these.
                                title = 'namepart'
                                val = 'deprel'
                                find = 'flat:name'
                                temp_ref.merge(self.find_val(title, val, temp_ref, start, deps, find))
                            if 'w2v_sim' in var_list:
                                title = 'w2v_sim'
                                temp_ref.merge(self.w2v(title, temp_ref, w2vmod, start, ref_ment))
                            if 'wnet_syn' in var_list:
                                title = 'wnet_syn'
                                var = 'lemma'
                                start_list = wnet_list
                                temp_ref.merge(self.in_list(title, var, temp_ref, start_list, ref_ment))



                        elif typ == 'pron': # pronoun specific vars
                            if 'person_match' in var_list:
                                title = 'person_match'
                                val = 'morph'
                                temp_ref.merge(self.compare(title, val, temp_ref, start, ref_ment)) 
                            if 'ptypemen' in var_list:
                                title = 'ptypemen'
                                val = 'deprel'
                                temp_ref.merge(self.get_val(title, val, temp_ref, start))

                        temp_ref = temp_ref.fillna(0)
                        all_refs.append(temp_ref)
            except IndexError:
                ret = True
        if ret == False and all_refs:
            all_refdf = pd.concat(all_refs)
            return all_refdf
        else:
            return 'skip'

            
    
    def set_up_vars(self, docid, row, deps, df, answers, var_list, typ, ids, w2vmod, train):#add docid
        '''
        set up variables adn dfs for creating vars.
        '''
        if train == True:
            start_refs = [deps[deps['shift'] == row.start_ref]] # can't be more than one, but program accepts lists here.
            cor_ref = row.cor
            incor_ref = row.incor
            refs = [cor_ref, incor_ref]
            tempdf = self.make_vars(deps, start_refs, refs, var_list, typ, w2vmod, train) # appends head ref pair variables
            if not type(tempdf) == str:
                df = pd.concat([df, tempdf])
                for start_ref in start_refs:
                    iddf_cor = pd.DataFrame({'docid': [docid], 'start': start_ref['shift'], 'ref':cor_ref})
                    iddf_incor = pd.DataFrame({'docid': [docid], 'start': start_ref['shift'], 'ref':incor_ref})
                    ids = ids.append(iddf_cor)
                    ids = ids.append(iddf_incor)
                    answers = answers.append(pd.DataFrame({'answer': [1]}))
                    answers = answers.append(pd.DataFrame({'answer': [0]}))
        elif train == False:
            ser = row.series
            if ser == True:#change morph for series to plural and split seperate mention for singular short ref.
                start_ment = deps[deps['shift'] == row.start_ref]
                start_ment_full = start_ment
                start_ment_full['morph'] = self.sing_to_plur(start_ment['morph'].values[0].split('|'))
                start_ment_part = start_ment
                start_ment_part['full_men'] = start_ment_part['part_men']
                start_ment_part['tk_shifts'] = start_ment_part['part_shifts']
                start_refs = [start_ment_full, start_ment_part]
            else:
                start_refs = [deps[deps['shift'] == row.start_ref]]
            refs = row.refs
            tempdf = self.make_vars(deps, start_refs, refs, var_list, typ, w2vmod, train)
            if not type(tempdf) == str:
                df = pd.concat([df, tempdf])
                c = 1
                sery = 'false'
                for start_ref in start_refs:
                    if len(start_refs) > 1: 
                        if c > 1:
                            sery = 'sing'
                        c += 1
                        for ref in refs:
                            iddf = pd.DataFrame({'docid': [docid], 'start': start_ref['shift'], 'ref':ref, \
                                                'series': [sery]})
                            ids = ids.append(iddf)   
                    for ref in refs:
                        iddf = pd.DataFrame({'docid': [docid], 'start': start_ref['shift'], 'ref':ref, \
                                            'series': [sery]})
                        ids = ids.append(iddf)
                               
        return df, answers, ids


                    
                    
    def start_vars(self, deps, answers, train, gold_var, no_gold_var, \
                   train_ids, idstr, ids_gold, \
                   ids_nogold, test_ids, train_var, gold, no_gold, type_all, docid, var_list, w2vmod):
        '''
        create mention variable lists for training and testing.
        '''
        
        if docid in train_ids: #make training data.
            print('train')
            traindoc = train[train['docid'] == docid]
            for i, trow in traindoc.iterrows():
                train_var, answers, idstr = self.set_up_vars(docid, trow, deps, train_var, answers, \
                                                                        var_list, type_all, idstr, w2vmod, train=True)
            
        elif docid in test_ids: # make test data
            print('test')
            no_answers = None
            golddoc = gold[gold['docid'] == docid]
            for i, rowg in golddoc.iterrows():
                gold_var, no_answers, ids_gold = self.set_up_vars(docid, rowg, deps, gold_var, no_answers, \
                                                var_list, type_all, ids_gold, w2vmod, train=False)

            

            nogolddoc = no_gold[no_gold['docid'] == docid]
            for i, rowng in nogolddoc.iterrows():
                no_gold_var, no_answers, ids_nogold = self.set_up_vars(docid, rowng, deps, no_gold_var, no_answers, \
                                                    var_list, type_all, ids_nogold, w2vmod, train=False)

        return answers, idstr, ids_gold, \
        ids_nogold, train_var, gold_var, no_gold_var, 
        
        
    def get_train_ids(self, train_per):
        '''
        randomizes training and test sets based on given percentage value, 
        returning the docids in two lists for training and testing.
        '''
        numlist = self.newdf.docid.values # all docid values to np array
        train_per = math.floor(train_per * len(numlist))
        train_ids = random.sample(numlist.tolist(), train_per) #harcoded quantity of texts.
        test_ids = [x for x in numlist if x not in train_ids]
        return train_ids, test_ids

    
    def create_vars(self, var_list, type_all, w2vmod, train_per, train_new = False, create_men=True):
        '''
        pass through manual sieves, collect mention pairs, and create final variable list for all 
        training and test data sets.
        '''
        if train_new == True:
            self.train_ids, self.test_ids = self.get_train_ids(train_per) #set ids fro training
        for i, nrow in self.newdf.iterrows():
            docname = nrow.docname
            print(docname)
            docid = nrow.docid
            
            if type_all == 'noun':
                mens = pd.read_csv('coref/men_1pass/{0}'.format(nrow.docname), delimiter='\t') # temp method for testing.
                deps = pd.read_csv('coref/1pass/{0}'.format(nrow.docname), delimiter='\t')
                
                
                
                self.train_noun, self.gold_noun, \
                self.no_gold_noun = self.create_candidates(mens, deps, self.train_noun, self.gold_noun, self.no_gold_noun, \
                                                           docid, self.train_ids, self.test_ids, type_all)
                
                self.answers_noun, self.idstr_noun, self.ids_ne_gold, self.ids_ne_nogold, \
                self.train_var_noun, self.gold_var_noun, \
                self.no_gold_var_noun = self.start_vars(deps, self.answers_noun, \
                     self.train_noun, self.gold_var_noun, self.no_gold_var_noun, \
                     self.train_ids, self.idstr_noun, self.ids_noun_gold, \
                     self.ids_noun_nogold, self.test_ids, self.train_var_noun, \
                     self.gold_noun, self.no_gold_noun, type_all, docid, var_list, w2vmod)
            
            
            elif type_all == 'ne':
                mens = pd.read_csv('coref/men_sieves/{0}'.format(nrow.docname), delimiter='\t') # temp method for testing.
                deps = pd.read_csv('coref/sieves/{0}'.format(nrow.docname), delimiter='\t')
                
                
                
                self.train_ne, self.gold_ne, \
                self.no_gold_ne = self.create_candidates(mens, deps, self.train_ne, self.gold_ne, self.no_gold_ne, \
                                                           docid, self.train_ids, self.test_ids, type_all)
            
            
                self.answers_ne, self.idstr_ne, self.ids_ne_gold, self.ids_ne_nogold, \
                self.train_var_ne, self.gold_var_ne, \
                self.no_gold_var_ne = self.start_vars(deps, self.answers_ne, \
                     self.train_ne, self.gold_var_ne, self.no_gold_var_ne, \
                     self.train_ids, self.idstr_ne, self.ids_ne_gold, \
                     self.ids_ne_nogold, self.test_ids, self.train_var_ne, \
                     self.gold_ne, self.no_gold_ne, type_all, docid, var_list, w2vmod)
                
                
            elif type_all == 'pron':
                mens = pd.read_csv('coref/men_2pass/{0}'.format(nrow.docname), delimiter='\t') # temp method for testing.
                deps = pd.read_csv('coref/2pass/{0}'.format(nrow.docname), delimiter='\t')
                
                
                
                self.train_pron, self.gold_pron, \
                self.no_gold_pron = self.create_candidates(mens, deps, self.train_pron, self.gold_pron, self.no_gold_pron, \
                                                           docid, self.train_ids, self.test_ids, type_all)

                self.answers_pron, self.idstr_pron, self.ids_ne_gold, self.ids_ne_nogold, \
                self.train_var_pron, self.gold_var_pron, \
                self.no_gold_var_pron = self.start_vars(deps, self.answers_pron, \
                     self.train_pron, self.gold_var_pron, self.no_gold_var_pron, \
                     self.train_ids, self.idstr_pron, self.ids_pron_gold, \
                     self.ids_pron_nogold, self.test_ids, self.train_var_pron, \
                     self.gold_pron, self.no_gold_pron, type_all, docid, var_listt, w2vmod)
            

            if not os.path.exists('coref/sets'): #for testing only
                os.mkdir('coref/sets')

            if type_all == 'ne':

                self.train_ne.to_csv('coref/sets/train_ne', sep='\t')
                self.gold_ne.to_csv('coref/sets/gold_ne', sep='\t') 
                self.no_gold_ne.to_csv('coref/sets/no_gold_ne', sep='\t')
                self.train_var_ne.to_csv('coref/train_var_ne', sep='\t')
                self.gold_var_ne.to_csv('coref/gold_var_ne', sep='\t')
                self.no_gold_var_ne.to_csv('coref/nogold_var_ne', sep='\t')
                self.answers_ne.to_csv('coref/answers_ne', sep='\t')
                self.idstr_ne.to_csv('coref/idstr_ne', sep='\t')
                self.ids_ne_gold.to_csv('coref/ids_ne_gold', sep='\t')
                self.ids_ne_nogold.to_csv('coref/ids_ne_nogold', sep='\t')

            if type_all == 'noun':

                self.train_noun.to_csv('coref/sets/train_noun', sep='\t')
                self.gold_noun.to_csv('coref/sets/gold_noun', sep='\t')
                self.no_gold_noun.to_csv('coref/sets/no_gold_noun', sep='\t')
                self.train_var_noun.to_csv('coref/train_var_noun', sep='\t')
                self.gold_var_noun.to_csv('coref/gold_var_noun', sep='\t')
                self.no_gold_var_noun.to_csv('coref/nogold_var_noun', sep='\t')
                self.answers_noun.to_csv('coref/answers_noun', sep='\t')
                self.idstr_noun.to_csv('coref/idstr_noun', sep='\t')
                self.ids_noun_gold.to_csv('coref/ids_noun_gold', sep='\t')
                self.ids_noun_nogold.to_csv('coref/ids_noun_nogold', sep='\t')

            if type_all == 'pron':

                self.train_pron.to_csv('coref/sets/train_pron', sep='\t')
                self.gold_pron.to_csv('coref/sets/gold_pron', sep='\t')
                self.no_gold_pron.to_csv('coref/sets/no_gold_pron', sep='\t')
                self.train_var_pron.to_csv('coref/train_var_pron', sep='\t')
                self.gold_var_pron.to_csv('coref/gold_var_pron', sep='\t')
                self.no_gold_var_pron.to_csv('coref/nogold_var_pron', sep='\t')
                self.answers_pron.to_csv('coref/answers_pron', sep='\t')
                self.idstr_pron.to_csv('coref/idstr_pron', sep='\t')
                self.ids_pron_gold.to_csv('coref/ids_pron_gold', sep='\t')    
                self.ids_pron_nogold.to_csv('coref/ids_pron_nogold', sep='\t')
                        
   

    def make_dummies(self, inpt):
            just_dummies = pd.get_dummies(inpt[['antecedenttype', 'deprolecomb', 'deproleant', 'deprolemen']])

            plusdummies = pd.concat([inpt, just_dummies], axis=1)
            plusdummies.drop(['antecedenttype', 'deprolecomb', 'deproleant', 'deprolemen'], \
                             inplace=True, axis=1)
            plusdummies = plusdummies.applymap(np.int)
            return plusdummies
    
    
    def fix_dummies(self, df, columns ):
        missing_cols = set( columns ) - set( df.columns ) # add columns from training
        for c in missing_cols:
            df[c] = 0
            
        extra_cols = set( df.columns ) - set( columns ) # subtract columns missning from training

        df = df[ columns ]
        return df

    
    def train(self, ttype):
        '''
        Train
        '''
        clf_gini = RandomForestClassifier()
        
        if ttype == 'noun':
            train_var = pd.read_csv('coref/noun_vars/train_var_noun', delimiter = '\t')
            answers = pd.read_csv('coref/noun_vars/answers_noun', delimiter = '\t')

            dummies = self.make_dummies(train_var)
            
            mod = clf_gini.fit(dummies, answers) 
            
        elif ttype == 'ne':
            train_var = pd.read_csv('coref/ne_vars/train_var_ne', delimiter = '\t')
            answers = pd.read_csv('coref/ne_vars/answers_ne', delimiter = '\t')

            dummies = self.make_dummies(train_var)
            mod = clf_gini.fit(dummies, answers)

            
        elif ttype == 'pron':
            train_var = pd.read_csv('coref/noun_vars/train_var_noun', delimiter = '\t')
            answers = pd.read_csv('coref/noun_vars/answers_noun', delimiter = '\t')

            dummies = self.make_dummies(train_var)
            
            mod = clf_gini.fit(dummies, answers)
        
        print(mod.score(dummies, answers))
        cols = dummies.columns
        return mod, cols
        
     
    def fix_df(self, no_gold_var, gold_var):
        no_gold_dummies = self.make_dummies(no_gold_var)
        gold_dummies = self.make_dummies(gold_var)

        no_gold_dummies = self.fix_dummies(no_gold_dummies, cols)
        gold_dummies = self.fix_dummies(gold_dummies, cols)
        return no_gold_dummies, gold_dummies

    
    def merge_dict(self, refs, ids):
        merge_dict = defaultdict(list)
        start = 0
        end = 0
        for testid in ids['docid'].unique():
            ids_frame = ids[ids['docid'] == testid]
            ids_new = ids_frame.values.tolist()
            ids_len = len(ids_new)
            end += ids_len
            ref_frame = refs[start:end]
            start = end
            doc_dict = defaultdict(list)
            ban_list = []
            c = 0
            for ref in ref_frame:
                c += 1
                if ref[1] == 1.0:
                    i = ids_new[c-1]
                    i1 = i[4]
                    i2 = i[2]
                    if not i2 in ban_list:
                        doc_dict[i1].append(i2)
                        ban_list.append(i2)
                        ban_list.append(i1)
            merge_dict[testid] = doc_dict
        return merge_dict
            
        
    def merge_ml(self, merge_dict, gold, ttype): 
        for docid, v in merge_dict:
            manual = True
            morph_merge = True
            men_dict = v
            if gold == True:
                mens = pd.read_csv('coref/men_sieves/{0}'.format(nrow.docname), delimiter='\t')
                self.merge(mens, men_dict, manual, morph_merge, gold)
                mens = pd.read_csv('coref/men_1pass/{0}'.format(nrow.docname), delimiter='\t') # temp method for testing.
                deps = pd.read_csv('coref/1pass/{0}'.format(nrow.docname), delimiter='\t')
                
            else:
                deps = pd.read_csv('coref/sieves/{0}'.format(nrow.docname), delimiter='\t')
                self.merge(mens, men_dict, manual, morph_merge, gold)
        
    
    
    
    def pipe(self, mod, ttype, cols):
        '''
        takes all models and pipes coresponding gold and no gold test sets through the model, 
        updating deps after each one.
        '''
        
        if ttype == 'ne':
            
            no_gold_var = pd.read_csv('coref/ne_vars/nogold_var_ne', delimiter = '\t')
            gold_var = pd.read_csv('coref/ne_vars/gold_var_ne', delimiter = '\t')
            ids_nogold = pd.read_csv('coref/ne_vars/ids_ne_nogold', delimiter = '\t')
            ids_gold = pd.read_csv('coref/ne_vars/ids_ne_gold', delimiter = '\t')
            
            no_gold_dummies, gold_dummies = self.fix_df(no_gold_var, gold_var)
            
            nogold = mod.predict(no_gold_dummies)
            gold = mod.predict(gold_dummies)
            
            nogold = mod.predict_proba(no_gold_dummies)
            gold = mod.predict_proba(gold_dummies)
            
            print(nogold)
#             self.merge_ml(nogold, ids_nogold, gold=True)
#             self.merge_ml(gold, ids_gold, gold=True)
            
            
            
            
        elif ttype == 'noun':
            
            no_gold_var = pd.read_csv('coref/noun_vars/nogold_var_noun', delimiter = '\t')
            gold_var = pd.read_csv('coref/noun_vars/gold_var_noun', delimiter = '\t')
            
            no_gold_dummies, gold_dummies = self.fix_df(no_gold_var, gold_var)
            
            nogold = mod.predict(no_gold_var)
            gold = mod.predict(gold_var)
            
        elif ttype == 'pron':
            
            no_gold_var = pd.read_csv('coref/pron_vars/nogold_var_pron', delimiter = '\t')
            gold_var = pd.read_csv('coref/pron_vars/gold_var_pron', delimiter = '\t')
            
            no_gold_dummies, gold_dummies = self.fix_df(no_gold_var, gold_var)
            
            nogold = mod.predict(no_gold_var)
            gold = mod.predict(old_var)
        
        
        
        
        
        
        
#         self, mens, men_dict, manual, morph_merge, gold

In [6]:
var_list = ['genderm', 'animacym', 'numberm', 'headm', 'partialheadm', 'sentdist', \
           'nsubjant', 'detant', 'nsubjmen', 'detmen', 'nar_match', 'antecedenttype', 'NEtype_match', \
           'person_match', 'namepart', 'clustagree', 'containnum', 'namepartref', 'deprolemen', 'deproleant', \
           'deprolecomb', 'samesent','ptypemen', 'w2v_sim', 'wnet_syn', 'shiftdist']

# 'sentinclust', 'clustlen', \
#            'clustpartial', 'clustw2v_sim', 'clustwnet_syn'] possible additions



example2 = Coreftrain()

example2.prepdf()

# w2vmod = KeyedVectors.load_word2vec_format('ruwikiruscorpora_0_300_20.bin.gz', binary=True, encoding='utf8')

# example2.create_vars(var_list, 'ne', w2vmod, train_per=.8, train_new = True) # first run


nemodel, cols = example2.train('ne')

example2.pipe(nemodel, 'ne', cols)

# example2.pair()


# example2.create_vars(var_list, 'noun', train_per=.8) # second run

# nounmodel = example2.train('noun')

# example2.pipe(nounmodel, 'noun')

# example2.pair()


# example2.create_vars(var_list, 'pron', train_per=.8) # third run

# pronmodel = example2.train('pron')

# example2.pipe(pronmodel, 'pron')

# example2.merge

FileNotFoundError: File b'coref/ne_vars/train_var_ne' does not exist